In [254]:
# %%writefile mine_wordnet.py
# from nltk.corpus import wordnet as wn
# from itertools import product,chain
# from collections import defaultdict
# import operator,json,csv,sys

# def get_definitions(word):
#     '''
#     Takes a word as input and returns all definitions in the form of a
#     dictionary with the synset as keys and definition text as values.
#     '''
#     definitions = defaultdict()
#     for i,j in enumerate(wn.synsets(word)):
#         definitions[j] = j.definition()
#     return definitions

# def get_ontology(seed_syn,word,write_file=False):
#     '''
#     Takes word as input and returns two dictionaries. One with the list
#     of hypernym synsets and one with a list of hyponym synsets.
#     '''
#     hypernyms = defaultdict()
#     hyponyms = defaultdict()
#     for i,j in enumerate(wn.synsets(word)):
#         hypernyms[j] = ",".join(list(chain(*[l.lemma_names() for l in j.hypernyms()])))
#         hyponyms[j] = ",".join(list(chain(*[l.lemma_names() for l in j.hyponyms()])))

#     hypo_words = [hyponyms.values()[i].split(',') for i in range(len(hyponyms.values())) if hyponyms.values()[i]][0]
#     hypo_syns = [get_definitions(hypo_words[i]).keys()[0] for i in xrange(len(hypo_words))]
#     relevant_hypo_synsets = calc_seed_similarity(seed_syn,hypo_syns)
    
#     # Write to file as feeder system to wolfram alpha api code
#     if write_file:
#         file_endpoint = "data/"+word+"_hyponyms.csv"
#         hypo_out = [str(relevant_hypo_synsets[i][0].lemma_names()[0]).replace('_',' ') for i in range(len(relevant_hypo_synsets))]        
#         print(hypo_out)         
# #             if i == 0:
# #                 with open(file_endpoint, 'w') as data_out:
# #                     csv.writer(hypo_out, data_out)
# #             else:
# #                 with open(file_endpoint, 'a+') as data_out:
# #                     csv.writer(hypo_out, data_out)
                
#         with open(file_endpoint, "w") as output:
#             writer = csv.writer(output, lineterminator='\n')
# #             for hypo in hypo_out:
#             writer.writerow(hypo_out) 
    
#     return hypernyms, relevant_hypo_synsets #hyponyms

# def calc_seed_similarity(seed_syn,topic_syns,threshold=.2):
#     '''
#     Using Wu-Palmer Similiarity compare a seed synset to a list of
#     topic synsets. If the similarity score is above the threshold
#     it is considered relevant and returned.
#     '''
#     similarity_scores = defaultdict()
#     for i in range(len(topic_syns)):
#         score = seed_syn.wup_similarity(topic_syns[i]) # Wu-Palmer Similarity
#         if score > threshold:
#             similarity_scores[topic_syns[i]] = score
#     return sorted(similarity_scores.items(), key=operator.itemgetter(1), reverse=True) 

# def create_flashcard(card_front,card_back,mode='w'):
#     '''
#     Create a flashcard with the front and back of the card as input
#     arguments. By default a new data file will be written, but the mode
#     can be set to 'a' or 'a+' to append to an existing file.
#     '''
#     flashcard = {"fcid": 1,
#                  "order": 0,
#                  "term": card_front,
#                  "definition": card_back,
#                  "hint": "",
#                  "example": "",
#                  "term_image": None,
#                  "hint_image": None}
    
#     # Save the data
#     file_endpoint = "data/"+card_front+".json"

#     if mode.startswith('a'):
#         with open(file_endpoint, 'a+') as data_out:
#             json.dump(',', data_out)
#             json.dump(flashcard, data_out)
#     else:
#         with open(file_endpoint, 'w') as data_out:
#             json.dump(flashcard, data_out) 

# #### TODO: Not yet converted to real function
# # def get_synonyms(word):
# #     print "THESAURUS"
# #     print 50 * '*'
# #     for i,j in enumerate(wn.synsets(topic)):
# #         print "Meaning",i, "NLTK ID:", j.name()
# #         print "Definition:",j.definition()
# #         print "Synonyms:",  ", ".join(j.lemma_names())
# #         print

# # Start by providing seed word to filter out unrelated topics/words
# SEED = 'math'
# seed_syn = get_definitions(SEED).keys()[0]
# # print 'SEED: ',SEED
# # print seed_syn

# topic = 'triangle'
# topic_syns = get_definitions(topic)
# # print 'TOPIC: ',topic
# # print 'SYNSETS: ',topic_syns

# relevant_synsets = calc_seed_similarity(seed_syn,topic_syns.keys())
# # for i in range(len(relevant_synsets)):
# #     create_flashcard(topic,topic_syns[relevant_synsets[i][0]],mode='a+')

# hypers,hypos = get_ontology(seed_syn,topic,write_file=True)

In [245]:
!python mine_wordnet.py

['oblique triangle', 'right triangle', 'obtuse triangle', 'acute triangle', 'wedge', 'equilateral triangle', 'scalene triangle', 'isosceles triangle']


In [243]:
hypo_out = ['oblique triangle', 'right triangle', 'obtuse triangle', 'acute triangle', 'wedge', 'equilateral triangle', 'scalene triangle', 'isosceles triangle']

for hypo in hypo_out:
    print(hypo)

oblique triangle
right triangle
obtuse triangle
acute triangle
wedge
equilateral triangle
scalene triangle
isosceles triangle


In [264]:
%%writefile get_flashcards.py

from flask import Flask

app = Flask(__name__)

@app.route("/<category>")
def get_flashcards(category):
    import subprocess
    p = subprocess.Popen("python mine_wordnet.py " + category, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    print output
    
if __name__ == '__main__':
    app.run()

Overwriting get_flashcards.py


In [1]:
from nltk.corpus import wordnet as wn
from itertools import product,chain
from collections import defaultdict
from get_data_from_wolfram import *
import get_data_from_wolfram as gdfw
import operator,json,csv,sys,subprocess

def get_definitions(word):
    '''
    Takes a word as input and returns all definitions in the form of a
    dictionary with the synset as keys and definition text as values.
    '''
    definitions = defaultdict()
    for i,j in enumerate(wn.synsets(word)):
        definitions[j] = j.definition()
    return definitions

def get_ontology(seed_syn,word,write_file=False):
    '''
    Takes word as input and returns two dictionaries. One with the list
    of hypernym synsets and one with a list of hyponym synsets.
    '''
    hypernyms = defaultdict()
    hyponyms = defaultdict()
    for i,j in enumerate(wn.synsets(word)):
        hypernyms[j] = ",".join(list(chain(*[l.lemma_names() for l in j.hypernyms()])))
        hyponyms[j] = ",".join(list(chain(*[l.lemma_names() for l in j.hyponyms()])))

    if not hyponyms.values():
        return [],[]
    
    hypo_words = [hyponyms.values()[i].split(',') for i in range(len(hyponyms.values())) if hyponyms.values()[i]][0]
    hypo_syns = [get_definitions(hypo_words[i].replace(' ','_')).keys()[0] for i in xrange(len(hypo_words))]
    relevant_hypo_synsets = calc_seed_similarity(seed_syn,hypo_syns)
    
    print hypo_syns,relevant_hypo_synsets
    
    # Write to file as feeder system to wolfram alpha api code
    if write_file:
        file_endpoint = "data/"+word+"_hyponyms.csv"
        hypo_out = [str(relevant_hypo_synsets[i][0].lemma_names()[0]).replace('_',' ') for i in range(len(relevant_hypo_synsets))]        
        # print(hypo_out)         
                
        with open(file_endpoint, "w") as output:
            writer = csv.writer(output, lineterminator='\n')
            writer.writerow(hypo_out) 
    
    return hypernyms, relevant_hypo_synsets

def calc_seed_similarity(seed_syn,topic_syns,threshold=.2):
    '''
    Using Wu-Palmer Similiarity compare a seed synset to a list of
    topic synsets. If the similarity score is above the threshold
    it is considered relevant and returned.
    '''
    similarity_scores = defaultdict()
    for i in range(len(topic_syns)):
        score = seed_syn.wup_similarity(topic_syns[i]) # Wu-Palmer Similarity
        if score > threshold:
            similarity_scores[topic_syns[i]] = score
    return sorted(similarity_scores.items(), key=operator.itemgetter(1), reverse=True) 

def create_flashcard(card_front,card_back,mode='w'):
    '''
    Create a flashcard with the front and back of the card as input
    arguments. By default a new data file will be written, but the mode
    can be set to 'a' or 'a+' to append to an existing file.
    '''
    flashcard = {"fcid": 1,
                 "order": 0,
                 "term": card_front,
                 "definition": card_back,
                 "hint": "",
                 "example": "",
                 "term_image": None,
                 "hint_image": None}
    
    # Save the data
    file_endpoint = "data/"+card_front+".json"

    if mode.startswith('a'):
        with open(file_endpoint, 'a+') as data_out:
            json.dump(',', data_out)
            json.dump(flashcard, data_out)
    else:
        with open(file_endpoint, 'w') as data_out:
            json.dump(flashcard, data_out) 

#### TODO: Not yet converted to real function
# def get_synonyms(word):
#     print "THESAURUS"
#     print 50 * '*'
#     for i,j in enumerate(wn.synsets(topic)):
#         print "Meaning",i, "NLTK ID:", j.name()
#         print "Definition:",j.definition()
#         print "Synonyms:",  ", ".join(j.lemma_names())
#         print

def main(argv):
    # Start by providing seed word to filter out unrelated topics/words
    SEED = 'math'
    seed_syn = get_definitions(SEED).keys()[0]
    # print 'SEED: ',SEED
    # print seed_syn

    topic = argv
    topic_syns = get_definitions(topic)
    print 'TOPIC: ',topic
    # print 'SYNSETS: ',topic_syns

    # clean this code: 
    relevant_synsets = calc_seed_similarity(seed_syn,topic_syns.keys())
    print 'REL SYNSETS: ',relevant_synsets
    # for i in range(len(relevant_synsets)):
    #     create_flashcard(topic,topic_syns[relevant_synsets[i][0]],mode='a+')

    _,_ = get_ontology(seed_syn,topic,write_file=True) # hypernyms,hyponyms

if __name__ == "__main__":
    _,topic = sys.argv
    main(topic)
   
    p = subprocess.Popen("python get_data_from_wolfram.py " + topic, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    print output

In [129]:
    # Start by providing seed word to filter out unrelated topics/words
    SEED = 'math'
    seed_syn = get_definitions(SEED).keys()[0]
    # print 'SEED: ',SEED
    # print seed_syn

    topic = "trigonometry"
    topic_syns = get_definitions(topic)
    print 'TOPIC: ',topic
    # print 'SYNSETS: ',topic_syns

    # clean this code: 
    relevant_synsets = calc_seed_similarity(seed_syn,topic_syns.keys())
    print 'REL SYNSETS: ',relevant_synsets
    # for i in range(len(relevant_synsets)):
    #     create_flashcard(topic,topic_syns[relevant_synsets[i][0]],mode='a+')

    _,_ = get_ontology(seed_syn,topic,write_file=True) # hypernyms,hyponyms

TOPIC:  trigonometry
REL SYNSETS:  [(Synset('trigonometry.n.01'), 0.9)]
[Synset('spherical_trigonometry.n.01'), Synset('triangulation.n.02')] [(Synset('spherical_trigonometry.n.01'), 0.8571428571428571), (Synset('triangulation.n.02'), 0.3333333333333333)]


In [137]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn

In [307]:
# user input
syllabus = open("/Users/Alexander/hacking_edu/sample_math_syllabus.txt", "r").read()
syllabus

'Mathematics 117, Trigonometry, helps students develop skills sufficiently to write and\nuse the definition of trigonometric functions; sketch the graph of the trigonometric\nfunctions; prove identities; solve trigonometric equations; learn and then apply the law\nof the sines and cosines; learn how to write a complex number in trigonometric form\nand find all the roots of a complex number; learn polar coordinates system and the\ngraphs of some simple equations in polar; learn about conic sections (rectangular &\npolar), vector (applications & operations), and the exponential and logarithmic\nfunctions with applications and modeling'

In [306]:
# user input
syllabus_2 = open("/Users/Alexander/hacking_edu/syllabus_2.txt", "r").read()
syllabus_2

'This course is intended for college-bound students. In this class, we will study angles,\ndegree and radian measurements, the six different trigonometric functions, graphing,\nidentities, solving equations, Law of Sines, Law of Cosines, and other trig. Related\napplications.\nChapters to be covered\n1. The trigonometric functions\n2. Acute angles and right triangles\n3. Radian measure and circular functions\n4. Graphs of circular functions\n5. Trigonometric identities\n7. Applications of trigonometry and vectors\n6. Inverse trigonometric functions and trigonometric equations?'

In [130]:
reg_tokens = nltk.tokenize.regexp_tokenize(syllabus, r'[\w+]+')

In [253]:
syllabus2_tokens = nltk.tokenize.regexp_tokenize(syllabus_2, r'[\w+]+')

In [194]:
# filter out stop words for syllabus.txt
unigrams = [word for word in reg_tokens if word.lower() not in stopwords.words()]
bigrams_tuples = [bigram for bigram in nltk.bigrams(tokens)]  

In [256]:
# filter out stop words for syllabus_2.txt
unigrams_2 = [word for word in syllabus2_tokens if word.lower() not in stopwords.words()]
bigrams_tuples_2 = [bigram for bigram in nltk.bigrams(syllabus2_tokens)]  

In [257]:
bigrams = [ " ".join(bigram)  for bigram in bigrams ]
bigrams_2 = [ " ".join(bigram)  for bigram in bigrams_tuples_2 ]

In [258]:
bigrams_2[0:10]

['This course',
 'course is',
 'is intended',
 'intended for',
 'for college',
 'college bound',
 'bound students',
 'students In',
 'In this',
 'this class']

In [196]:
bigrams[0:5]

['Mathematics 117',
 '117 Trigonometry',
 'Trigonometry helps',
 'helps students',
 'students develop']

In [259]:
unigrams[0:10]

['Mathematics',
 '117',
 'Trigonometry',
 'helps',
 'students',
 'develop',
 'skills',
 'sufficiently',
 'write',
 'use']

In [157]:
import numpy as np
import pandas as pd

In [291]:
def split_lexicon_keywords(lexicon):
    lst = []
    for keyword in lexicon:
        for token in keyword.split(" "):
            if token not in stopwords.words():
                lst.append(token)
    return lst

In [236]:
def to_lowercase(math_list):
    # lower case all math words 
    word_list = []
    for word in math_list:
        try:
            word_list.append(word.lower())
        except: AttributeError
    return word_list

In [227]:
df_cal = pd.read_csv("/Users/Alexander/hacking_edu/calculus_lexicon.csv", header=None)
df_alg = pd.read_csv("/Users/Alexander/hacking_edu/algebra_lexicon.csv", header=None)
df_trig = pd.read_csv("/Users/Alexander/hacking_edu/trigonometry_lexicon.csv", header=None)
df_geo = pd.read_csv("/Users/Alexander/hacking_edu/geometry_lexicon.csv", header=None)

In [235]:
cal = df_cal[df_cal.columns].values[0]
alg = df_alg[df_alg.columns].values[0]
trig = df_trig[df_trig.columns].values[0]
geo = df_geo[df_geo.columns].values[0]

In [294]:
cal  = split_lexicon_keywords(cal)
alg  = split_lexicon_keywords(alg)
trig = split_lexicon_keywords(trig)
geo  = split_lexicon_keywords(geo)

/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [295]:
cal  = to_lowercase(cal)
alg  = to_lowercase(alg)
trig = to_lowercase(trig)
geo  = to_lowercase(geo)

In [308]:
# syllabus.txt
bigram_keywords = [word for word in bigrams if word in cal or word in alg or word in trig or word in geo]
unigram_keywords = [word for word in unigrams if word in cal or word in alg or word in trig or word in geo]

In [309]:
# syllabus_2.txt
bigram_keywords_2 = [word for word in bigrams_2 if word in cal or word in alg or word in trig or word in geo]
unigram_keywords_2 = [word for word in unigrams_2 if word in cal or word in alg or word in trig or word in geo]

In [310]:
# syllabus.txt
bigram_keywords

[]

In [311]:
# syllabus.txt
unigram_keywords

['definition',
 'functions',
 'graph',
 'functions',
 'identities',
 'solve',
 'equations',
 'law',
 'sines',
 'cosines',
 'complex',
 'number',
 'form',
 'complex',
 'number',
 'polar',
 'coordinates',
 'system',
 'simple',
 'equations',
 'polar',
 'conic',
 'sections',
 'rectangular',
 'polar',
 'vector',
 'operations',
 'exponential',
 'logarithmic',
 'functions']

In [312]:
# syllabus_2.txt
bigram_keywords_2

[]

In [313]:
# syllabus_2.txt
unigram_keywords_2

['bound',
 'angles',
 'degree',
 'radian',
 'functions',
 'identities',
 'equations',
 'trig',
 'functions',
 'angles',
 'right',
 'triangles',
 'measure',
 'circular',
 'functions',
 'circular',
 'functions',
 'identities',
 'trigonometry',
 'functions',
 'equations']

#More Signal 

    The math lexicons are able to pull out more specific math keywords than the wordnet is capable of doing.
    We can add a feature to the app -- copy and paste your syllabus math course description and SmartCards will generate flash cards for you.